In [2]:
pip install tensorflow

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import LambdaCallback

# Introducción
En este proyecto, utilizamos un dataset de imágenes obtenido de la librería TensorFlow, ya que las imágenes de Kaggle son demasiado numerosas y pesadas para este análisis. El dataset CIFAR-10 clasifica imágenes en 10 categorías: aviones, autos, pájaros, gatos, venados, perros, ranas, caballos, barcos y camiones. Para mejorar la precisión en la clasificación de estas imágenes, aplicamos Transfer Learning, seleccionando el modelo VGG16, preentrenado en el conjunto de datos ImageNet, que incluye millones de imágenes. Al reutilizar las capas iniciales de este modelo, que ya han aprendido a identificar características visuales generales como bordes y texturas, podemos aprovechar este conocimiento para clasificar con mayor precisión las imágenes de CIFAR-10. También se verá el impacto de usar tanto el Data Augmentation con el Tranfer Learning.
# Objetivos del Notebook
- Comparar los resultados de los modelos de Data Augmentation con los de tranfer learning, dividiendolos según que se usa
- Ver como son los modelos ya preentrenados de Tranfer Learning
- Y interpretar los resultados de cada uno de los modelos según su forma de entrenamiento


In [3]:
# Cargar el dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalizar las imágenes a valores entre 0 y 1 para que se pueda hacer el entrenamiento y pruebas en el modelo, luego de está limpieza
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [4]:
# En el dataset se puede encontro 50000 imágenes que están distribuidas en las 10 categorías anteriores de 32x32 y a color
print(train_images.shape)

(50000, 32, 32, 3)


In [5]:
# Crear un modelo
model = models.Sequential()

# Primera capa convolucional: 32 filtros de 3x3, activación ReLU, con imágenes de entrada de 32x32x3
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Segunda capa convolucional con 64 filtros
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Tercera capa convolucional con 64 filtros
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Aplanar la salida para las capas densas
model.add(layers.Flatten())

# Agregar una capa densa con 64 neuronas
model.add(layers.Dense(64, activation='relu'))

# Capa de salida con 10 neuronas (correspondiente a las 10 clases de CIFAR-10), usando softmax
model.add(layers.Dense(10, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Crear un callback para mostrar precisión y error por época
metrics_callback = LambdaCallback(on_epoch_end=lambda epoch, logs:
                                  print(f'Epoch {epoch + 1}: Precisión = {logs["accuracy"] * 100:.2f}%, Error = {(1 - logs["accuracy"]) * 100:.2f}%'))

# Entrenar el modelo sin Data Augmentation
print("Entrenamiento sin Data Augmentation")
history_no_augmentation = model.fit(train_images, train_labels,
                                    epochs=10,
                                    batch_size=64,
                                    callbacks=[metrics_callback])

# Evaluar el modelo sin Data Augmentation
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'\nTest accuracy sin Data Augmentation: {test_acc}')

# Guardar los pesos iniciales del modelo antes de reiniciarlo
initial_weights = model.get_weights()

# Reiniciar los pesos del modelo
model.set_weights(initial_weights)

Entrenamiento sin Data Augmentation
Epoch 1/10
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.3331 - loss: 1.8100Epoch 1: Precisión = 41.71%, Error = 58.29%
782/782 ━━━━━━━━━━━━━━━━━━━━ 64s 80ms/step - accuracy: 0.3334 - loss: 1.8095
Epoch 2/10
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.5442 - loss: 1.2794Epoch 2: Precisión = 56.08%, Error = 43.92%
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 79ms/step - accuracy: 0.5443 - loss: 1.2793
Epoch 3/10
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.6131 - loss: 1.1000Epoch 3: Precisión = 62.18%, Error = 37.82%
782/782 ━━━━━━━━━━━━━━━━━━━━ 60s 77ms/step - accuracy: 0.6131 - loss: 1.0999
Epoch 4/10
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.6533 - loss: 0.9919Epoch 4: Precisión = 65.97%, Error = 34.03%
782/782 ━━━━━━━━━━━━━━━━━━━━ 59s 75ms/step - accuracy: 0.6533 - loss: 0.9919
Epoch 5/10
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.6816 - loss: 0.9040Epoch 5: Precisión = 68.86%, Error = 31.14%
782/78

In [7]:
# Crear un generador de imágenes con data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,       # Rotar las imágenes aleatoriamente hasta 20 grados
    width_shift_range=0.2,   # Desplazamiento horizontal de hasta el 20% de la imagen
    height_shift_range=0.2,  # Desplazamiento vertical de hasta el 20% de la imagen
    horizontal_flip=True,    # Voltear horizontalmente las imágenes
)

# Ajustar el generador de datos al conjunto de entrenamiento
datagen.fit(train_images)

In [8]:
# Entrenar el modelo con Data Augmentation
print("Entrenamiento con Data Augmentation")
history_with_augmentation = model.fit(datagen.flow(train_images, train_labels, batch_size=64),
                                      steps_per_epoch=len(train_images) // 64,
                                      epochs=10,
                                      validation_data=(test_images, test_labels),
                                      callbacks=[metrics_callback])

# Evaluar el modelo con Data Augmentation
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'\nTest accuracy con Data Augmentation: {test_acc}')

Entrenamiento con Data Augmentation
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.5623 - loss: 1.2432Epoch 1: Precisión = 57.75%, Error = 42.25%
781/781 ━━━━━━━━━━━━━━━━━━━━ 93s 118ms/step - accuracy: 0.5624 - loss: 1.2432 - val_accuracy: 0.6817 - val_loss: 0.9159
Epoch 2/10
  1/781 ━━━━━━━━━━━━━━━━━━━━ 48s 62ms/step - accuracy: 0.5781 - loss: 0.9421

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Epoch 2: Precisión = 57.81%, Error = 42.19%
781/781 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.5781 - loss: 0.9421 - val_accuracy: 0.6738 - val_loss: 0.9433
Epoch 3/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.5996 - loss: 1.1350Epoch 3: Precisión = 60.33%, Error = 39.67%
781/781 ━━━━━━━━━━━━━━━━━━━━ 140s 121ms/step - accuracy: 0.5996 - loss: 1.1350 - val_accuracy: 0.6534 - val_loss: 1.0102
Epoch 4/10
  1/781 ━━━━━━━━━━━━━━━━━━━━ 49s 64ms/step - accuracy: 0.5938 - loss: 1.1047Epoch 4: Precisión = 59.38%, Error = 40.62%
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5938 - loss: 1.1047 - val_accuracy: 0.6595 - val_loss: 0.9809
Epoch 5/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.6120 - loss: 1.0980Epoch 5: Precisión = 61.59%, Error = 38.41%
781/781 ━━━━━━━━━━━━━━━━━━━━ 95s 121ms/step - accuracy: 0.6120 - loss: 1.0980 - val_accuracy: 0.6395 - val_loss: 1.0814
Epoch 6/10
  1/781 ━━━━━━━━━━━━━━━━━━━━ 47s 61ms/step - accuracy: 0.7031 - loss: 0.9108E

In [9]:
# Cargar el modelo preentrenado VGG16 sin las capas superiores
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False  # Congelar las capas

# Crear un modelo con las capas adicionales
model_transfer = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

# Compilar el modelo
model_transfer.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

# Entrenar el modelo
print("\nEntrenamiento del modelo CON Transfer Learning (SIN Data Augmentation):")
model_transfer.fit(train_images, train_labels, epochs=10, batch_size=64, callbacks=[metrics_callback])

# Evaluar el modelo
test_loss_transfer, test_acc_transfer = model_transfer.evaluate(test_images, test_labels)
print(f'\nTest accuracy (CON Transfer Learning SIN Data Augmentation): {test_acc_transfer}')

# Entrenar el modelo transfer learning con data augmentation
print("\nEntrenamiento del modelo CON Transfer Learning y CON Data Augmentation:")
model_transfer.fit(datagen.flow(train_images, train_labels, batch_size=64),
                   epochs=10,
                   validation_data=(test_images, test_labels),
                   callbacks=[metrics_callback])

# Evaluar el modelo
test_loss_transfer_aug, test_acc_transfer_aug = model_transfer.evaluate(test_images, test_labels)
print(f'\nTest accuracy (CON Transfer Learning y CON Data Augmentation): {test_acc_transfer_aug}')

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Entrenamiento del modelo CON Transfer Learning (SIN Data Augmentation):
Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 700ms/step - accuracy: 0.3973 - loss: 1.7092Epoch 1: Precisión = 46.86%, Error = 53.14%
782/782 ━━━━━━━━━━━━━━━━━━━━ 550s 700ms/step - accuracy: 0.3974 - loss: 1.7089
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 691ms/step - accuracy: 0.5431 - loss: 1.3104Epoch 2: Precisión = 54.56%, Error = 45.44%
782/782 ━━━━━━━━━━━━━━━━━━━━ 555s 691ms/step - accuracy: 0.5431 - loss: 1.3103
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 691ms/step - accuracy: 0.5635 - loss: 1.2484Epoch 3: Precisión = 56.53%, Error = 43.47%
782/782 ━━━━━━━━━━━━━━━━━━━━ 541s 691ms/step - accuracy: 0.5635 - loss: 1.2483
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 692ms/step - accuracy: 0.5765 - loss: 1.2120Epoch 4: Precisión = 57.66%, Error = 42.34%
782/782 ━━━━━━━━━━━━━━━━━━━━ 541s 692ms/step - accuracy: 0.5765 - loss: 1.2120
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 725ms/step - accuracy: 0.5014 - loss: 1.4289Epoch 1: Precisión = 50.60%, Error = 49.40%
782/782 ━━━━━━━━━━━━━━━━━━━━ 676s 865ms/step - accuracy: 0.5014 - loss: 1.4289 - val_accuracy: 0.5867 - val_loss: 1.1852
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 721ms/step - accuracy: 0.5164 - loss: 1.3776Epoch 2: Precisión = 51.58%, Error = 48.42%
782/782 ━━━━━━━━━━━━━━━━━━━━ 672s 859ms/step - accuracy: 0.5164 - loss: 1.3776 - val_accuracy: 0.5816 - val_loss: 1.1841
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 721ms/step - accuracy: 0.5139 - loss: 1.3787Epoch 3: Precisión = 51.89%, Error = 48.11%
782/782 ━━━━━━━━━━━━━━━━━━━━ 716s 903ms/step - accuracy: 0.5139 - loss: 1.3787 - val_accuracy: 0.5815 - val_loss: 1.1823
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 722ms/step - accuracy: 0.5195 - loss: 1.3664Epoch 4: Precisión = 51.92%, Error = 48.08%
782/782 ━━━━━━━━━━━━━━━━━━━━ 708s 858ms/step - accuracy: 0.5195 - loss: 1.3664 - val_accuracy: 0.5866 - val_loss: 1.1673
Epo

# Conclusión e interpretación
Los modelos sacaron los siguientes resultados finales en los datos de prueba: Modelo sin Data Augmentation y sin Transfer Learning dio igual a 70.90% de precisión, Modelo con Data Augmentation y sin Transfer Learning dio una precisión de 69.05%, Modelo sin Data Augmentation y con Transfer Learning fue de 61.28% y el último Modelo con Data Augmentation y con Transfer Learning dio un resultado de precisión de 58.37%.

En la explicación, en el caso del Data Augmentation, ya se comentó que esta técnica es muy útil en el caso de imágenes que tengan similitudes muy notables, como tal vez las razas de perros, pero en el caso de este dataset, las categorías son 10 y muy distintas entre cada una, por lo que generar más ejemplos de los que ya tenemos puede resultar contraproducente a la hora de entrenar el modelo. Aun así, sin meter el Transfer Learning, estos sin ninguna de las dos técnicas y solo con el Data Augmentation dieron muy buenos resultados, con una variación muy baja a la hora de la prueba, por lo que los modelos en este caso se mantuvieron bien.

Ya cuando incluimos el Transfer Learning se puede ver una disminución muy grande en la precisión final del modelo en los resultados finales, y esto puede ser por el hecho de que el modelo que escogimos no estaba entrenado con todas las categorías del CIFAR-10, por lo que al ser tantas categorías puede que sea muy preciso para ciertas imágenes, pero muy malo para otras. Esto puede explicar su bajo rendimiento. Lo mismo sucede cuando agregamos el Data Augmentation a la ecuación, ya que desde un inicio el modelo está mal; esta técnica lo que hace es confundirlo más, por lo que da los malos resultados que se nos presentaron anteriormente en ambos casos.

Por lo que en el caso del Transfer Learning es bueno utilizarlo, pero en este caso capaz se escogió un modelo ya preentrenado que no contaba con la información suficiente de las categorías que se querían utilizar, por lo que se confundió más y ya no hablemos de agregarle el Data Augmentation. Por lo que sí se vio cómo funciona el Transfer Learning, solo que no se supo apreciar bien en sus resultados de precisión.